<a href="https://colab.research.google.com/github/ahmed98Osama/Analysis-of-US-Home-Prices-and-Median-Income-Dashboard/blob/main/Analusis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install pandas
!pip install gunicorn
!pip install jupyter_dash

     |████████████████████████████████| 9.6 MB 14.7 MB/s 
     |████████████████████████████████| 357 kB 74.1 MB/s 
     |████████████████████████████████| 79 kB 4.3 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=f6d7ce5446d9deaacd4d5907aa0e4177425d06af99a8de4b1e23d005c681dde3
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd  /content/drive/MyDrive/Data

Mounted at /content/drive
/content/drive/MyDrive/Data


In [ ]:

import dash
import dash_table
from dash import dash_table as dt
from dash.dash_table.Format import Group

# import dash_core_components as dcc
# import dash_html_components as html
from dash import dcc
from dash import html

import jupyter_dash
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Output, Input


import pandas as pd


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# --- load data ---

df_h = pd.read_csv('/content/drive/MyDrive/Data/df_h.csv')
df_h['Date'] = pd.to_datetime(df_h['Date'])

df_t = pd.read_csv('/content/drive/MyDrive/Data/dash_table.csv')


df_arima = pd.read_csv('/content/drive/MyDrive/Data/df_arima.csv')
df_arima['Date'] = pd.to_datetime(df_arima['Date'])
df_arima['Date'] = df_arima['Date'].dt.strftime('%Y-%m')

options = []
for column in df_h.columns:
    options.append({'label': '{}'.format(column, column),
                   'value': column})
options = options[1:]


# #####
# sales = pd.read_csv('train.csv')
# sales['Order Date'] = pd.to_datetime(sales['Order Date'])
# sales['Year'] = sales['Order Date'].dt.year
# sales['Month'] = sales['Order Date'].dt.month_name()
# #####

In [ ]:


external_stylesheets = ['/content/drive/MyDrive/Data/assets/style.css' , '/content/drive/MyDrive/Data/assets/s1.css']

app = \
    jupyter_dash.JupyterDash(__name__,external_stylesheets=external_stylesheets,meta_tags=[{"name": "viewport", "content": "width=device-width"}])
server = app.server

app.layout = html.Div((

    html.Div([
        html.Div([
            html.Div([
                html.H2('Analysis of US Home Prices and Median Income', style = {'margin-bottom': '0px', 'color': 'white'}),
            ])
        ], className = "one-half column", id = "title1"),

        html.Div([
            html.P('''An interactive dashboard displaying data for income, 
                       population and housing prices in the United States. ''', className = 'fix_label', style = {'color': 'white'}),

        ], className = "one-half column", id = "title2"),

        html.Div([
            html.P('By Ahmed Osama & Islam Waheed', className = 'fix_label', style = {'color': 'white'}),

        ], className = "one-third column", id = 'title3'),

    ], id = "header", className = "row flex-display", style = {"margin-bottom": "25px"}),

    html.Div([
              html.Div([html.Label('Select a city to see home price data',className = 'fix_label', style = {'color': 'white'}),
                    dcc.Dropdown(  # app.layout.update_layout(paper_bgcolor="black")
                    id='city-dropdown',
                    options=options,
                    value='New York, NY' , #New York, NY , Denver, CO
                    multi=False,
                    clearable=True,
                    searchable=True,
                    placeholder='Choose a City...',
                    ) ,         html.Div([
                                  html.Div(id='text1'),
                                  html.Div(id='text2'),

                               ], className = 'create_container2 four columns', style = {'width': '500px' ,  'border-radius': '5px',   'background-color': '#F9F9F9',   'margin': '10px',   'padding': '15px',   'position': 'relative',   'box-shadow': '2px 2px 2px lightgrey' }),
                        html.Div([

                                  html.Div(id='text3'),
                                  html.Div(id='text4'),

                               ], className = 'create_container2 four columns', style = {'width': '500px' ,  'border-radius': '5px',   'background-color': '#F9F9F9',   'margin': '10px',   'padding': '15px',   'position': 'relative',   'box-shadow': '2px 2px 2px lightgrey' }),
                    ] , className = 'create_container2 four columns', style = {'height': '500px' }),  
        html.Div([
            html.Div(dcc.Graph(id='forecast-container',)),

        ], className = 'create_container2 eight columns', style = {'height': '500px'}),



    ], className = "row flex-display"),

    html.Div((
        html.Div([html.Label('''Filter data in the first row of the table to see 
                       changes on the map and graph!Example : " > 100000" in Population col ,
                      " < 300000 " in Median Home price col , etc.''',
                 style={'textAlign': 'center','padding': '5px' ,'color': 'white' }),
            dt.DataTable(id = 'datatable',
                         columns = [{'name': i, 'id': i,'deletable': False,
        'selectable': True,
        'hideable': False,} for i in df_t.columns],
                         # page_action='native',
                         data=df_t.to_dict('records'),
                         page_size=50,
                         filter_action='native',
                         editable=False,
                         sort_action = "native",
                         sort_mode = "multi",
                         row_deletable=True,
                         selected_rows=[],
                         page_current=0,
                         # column_selectable="single",
                         # fill_width=False,
                         style_table={
                         #"width": "100%",
                                  "height": "100%"},
                         virtualization = True,
                         style_cell = {'textAlign': 'center',
                                       'min-width': '100px',
                                       'backgroundColor': 'white',
                                       'color': '#1f2c56',
                                       'border-bottom': '0.01rem solid #19AAE1',
                                       },
                         style_as_list_view = True,
                         style_cell_conditional=[{'if': {'column_id': c},
                            'textAlign': 'center', 'textsize' : 20 ,'color': 'green' , } for c in
                            ['Population','City' , 'Lat' , 'Long',
                            'Income($)',
                            'Price']],
                         style_header = {
                             'backgroundColor': '1f2c56#',
                             'fontWeight': 'bold',
                             'font': 'Lato, sans-serif',
                             'color': 'orange',
                             'border': '#1f2c56',
                         },
                         
                         style_data = {'textOverflow': 'hidden', 'color': '1f2c56'},
                         fixed_rows = {'headers': True},
                         )

        ], className = 'create_container2 five columns'),



        html.Div([
            dcc.Graph(id='graph-container',style={'padding': '25px'},config = {'displayModeBar': 'hover'}),
            dcc.Graph(id='map-container',style={'padding': '25px'},config = {'displayModeBar': 'hover'})], 
            className = 'create_container2 seven columns'),

            
            ), className = "row flex-display"),

), id= "mainContainer", style={"display": "flex", "flex-direction": "column"})




In [ ]:
# --- dropdown callback ---


@app.callback(Output('forecast-container', 'figure'),
              [Input('city-dropdown', 'value')])
def update_figure(selected_city):
    dff = df_h[['Date', selected_city]]

    # dff[selected_city] = dff[selected_city].round(0)

    dfa = df_arima[df_arima['City'] == selected_city]

    fig = px.line(dff,
                  x='Date',
                  y=selected_city,
                  hover_data={
                      selected_city: ':$,f'
                  }).update_traces(line=dict(color='#1f3b4d', width=5))

    fig.update_layout(
        template='xgridoff',
        yaxis={'title': 'Median Home Price ($USD)'},
        xaxis={'title': 'Year'},
        title={
            'text': 'Median Home Price vs. Year for {}'.format(selected_city),
            'font': {
                'size': 14
            },
            'x': 0.5,
            'xanchor': 'center',
        })

    return fig

# --- dropdown callback ---


@app.callback(Output('text1', 'children'),
              [Input('city-dropdown', 'value')])
def update_Bans(selected_city):
    dff = df_h[['Date', selected_city]]


    dfa = df_arima[df_arima['City'] == selected_city]

    dff = df_h[['Date', selected_city]]

    min =dff[selected_city].min()
    roundmin =round(min)

    dff[dff[selected_city]==min]['Date']


    max =dff[selected_city].max()
    roundmax =round(max)

    dff[dff[selected_city]==max]['Date']


    return [

               html.H6(children = 'Min Price',
                       style = {'textAlign': 'center',
                                'color': 'white'}
                       ),

               html.P('${0}'.format(roundmin),
                      style={'textAlign': 'center',
                             'color': '#19AAE1',
                             'fontSize': 15,
                             'margin-top': '-10px'
                             }
                      ),
    ]

# --- graph callback ---
###############################
@app.callback(Output('text2', 'children'),
              [Input('city-dropdown', 'value')])
def update_Bans(selected_city):
    dff = df_h[['Date', selected_city]]


    dfa = df_arima[df_arima['City'] == selected_city]

    dff = df_h[['Date', selected_city]]

    min =dff[selected_city].min()
    roundmin =round(min)

    minyear = dff[dff[selected_city]==min]['Date']


    max =dff[selected_city].max()
    roundmax =round(max)

    dff[dff[selected_city]==max]['Date']


    minyear.iloc[0]

    dt_nobj = datetime.date(maxyear.iloc[0]).strftime('%d-%m-%Y')


    return [

               html.H6(children = 'Year ',
                       style = {'textAlign': 'center',
                                'color': 'white'}
                       ),

               html.P('{0}'.format(dt_nobj),
                      style={'textAlign': 'center',
                             'color': '#19AAE1',
                             'fontSize': 15,
                             'margin-top': '-10px'
                             }
                      ),
    ]
#######################
# --- dropdown callback ---


@app.callback(Output('text3', 'children'),
              [Input('city-dropdown', 'value')])
def update_Bans(selected_city):
    dff = df_h[['Date', selected_city]]


    dfa = df_arima[df_arima['City'] == selected_city]

    dff = df_h[['Date', selected_city]]

    min =dff[selected_city].min()
    roundmin =round(min)

    dff[dff[selected_city]==min]['Date']


    max =dff[selected_city].max()
    roundmax =round(max)

    dff[dff[selected_city]==max]['Date']
    



    return [

               html.H6(children = 'Max Price',
                       style = {'textAlign': 'center',
                                'color': 'white'}
                       ),

               html.P('${0}'.format(roundmax),
                      style={'textAlign': 'center',
                             'color': 'red',
                             'fontSize': 15,
                             'margin-top': '-10px'
                             }
                      ),
    ]

# --- graph callback ---
###############################
@app.callback(Output('text4', 'children'),
              [Input('city-dropdown', 'value')])
def update_Bans(selected_city):
    dff = df_h[['Date', selected_city]]


    dfa = df_arima[df_arima['City'] == selected_city]

    dff = df_h[['Date', selected_city]]

    min =dff[selected_city].min()
    roundmin =round(min)

    minyear = dff[dff[selected_city]==min]['Date']


    max =dff[selected_city].max()
    roundmax =round(max)

    maxyear = dff[dff[selected_city]==max]['Date']

    maxyear.iloc[0]

    dt_objM = datetime.date(maxyear.iloc[0]).strftime('%d-%m-%Y')


    return [

               html.H6(children = 'Year ',
                       style = {'textAlign': 'center',
                                'color': 'white'}
                       ),

               html.P('{0}'.format(dt_objM),
                      style={'textAlign': 'center',
                             'color': 'red',
                             'fontSize': 15,
                             'margin-top': '-10px'
                             }
                      ),
    ]
#########################

@app.callback(Output('graph-container', 'figure'),
              [Input('datatable', 'derived_virtual_data')])
def update_scatter(all_rows_data):
    dff = pd.DataFrame(all_rows_data)

    fig = px.scatter(
        dff,
        x='Price',
        y='Income($)',
        size='Population',
        size_max=25,
        color='Price',
        color_continuous_scale='jet',
        hover_name='City',
        hover_data={
            'Lat': False,
            'Long': False,
            'Population': ':,2f',
            'Income($)': ':$,2f',
            'Price': ':$,2f',
        },
    )

    fig.update_layout(
        title_text=
        'Median  Income vs. Median Home Price in the United States',
        title_font_size=14,
        title_xref='container',
        title_y=0.95,
        title_x=0.5,
        showlegend=False,
        width=900,
        #height=500,
        template='xgridoff',
    )

    return fig


# --- map callback ---


@app.callback(Output('map-container', 'figure'),
              [Input('datatable', 'derived_virtual_data')])
def update_map(all_rows_data):
    dff = pd.DataFrame(all_rows_data)

    fig = px.scatter_geo(
        dff,
        lat='Lat',
        lon='Long',
        size='Population',
        size_max=15,
        color='Price',
        color_continuous_scale='jet',
        hover_name='City',
        hover_data={
            'Lat': False,
            'Long': False,
            'Population': ':,2f',
            'Income($)': ':$,2f',
            'Price': ':$,2f',
        },
    )

    fig.update_layout(
        title_text=
        'Median Home Price (2020), Income (2017) and City Populations (2019) for the United States',
        title_font_size=14,
        title_xref='container',
        title_y=0.9,
        title_x=0.5,
        #width=900,
        #height=500,
        showlegend=True,
        geo=dict(scope='usa', landcolor='rgba(225, 225, 225, 0.75)'),
        legend_title_text='City size, ranked by population',
    )

    return fig

app.run_server(debug=True)


Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
from datetime import datetime
selected_city = 'Chicago, IL'
dff = df_h[['Date', selected_city]]

selected_city
min =dff[selected_city].min()
roundmin =round(min)

dff[dff[selected_city]==min]['Date']


max =dff[selected_city].max()
roundmax =round(max)

maxyear = dff[dff[selected_city]==max]['Date']
maxyear.iloc[0]

dt_obj = datetime.date(maxyear.iloc[0]).strftime('%d-%m-%Y')

print(dt_obj)

31-01-2007
